<a href="https://colab.research.google.com/github/spkkarri/Tools/blob/main/Journals_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests pandas
#openalex cors scholarly

In [ ]:
import requests
import pandas as pd
import time
Search_qwuery = "Quantum Algorithm foundations for Machine learning"
def reconstruct_abstract(inverted_index):
    if not isinstance(inverted_index, dict):
        return ""
    # Flatten and collect all (position, word) pairs
    words = []
    for word, positions in inverted_index.items():
        for pos in positions:
            words.append((pos, word))
    # Sort by position and join
    words.sort()
    return " ".join([w for _, w in words])



def search_openalex_all_pages(query, per_page=200):
    """
    Search OpenAlex API for all works matching a query in title and abstract.

    Args:
        query (str): Search string.
        per_page (int): Number of results per page (max 200).

    Returns:
        pd.DataFrame: DataFrame with columns 'title' and 'abstract'.
    """
    base_url = "https://api.openalex.org/works"
    params = {
        "per_page": per_page,
        "sort": "relevance_score:desc",
        "filter": f"title_and_abstract.search:{query.replace(' ', '+')}"
    }

    records = []
    page = 1
    while True:
        params["page"] = page
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        # Stop if no more results
        if not data.get("results"):
            break

        for result in data.get("results", []):
            title = result.get("title", "")
            abstract = result.get("abstract_inverted_index", "")
            abstract_text = reconstruct_abstract(abstract)
            records.append({"title": title, "abstract": abstract_text})

        page += 1

    df = pd.DataFrame(records)
    return df


In [ ]:
import requests
import pandas as pd
import os


def search_openalex(query, per_page=10, page=1):
    """
    Search OpenAlex API for works matching a query in title and abstract.

    Args:
        query (str): Search string.
        per_page (int): Number of results per page.
        page (int): Page number.

    Returns:
        pd.DataFrame: DataFrame with columns 'title' and 'abstract'.
    """
    base_url = "https://api.openalex.org/works"
    params = {
        "page": page,
        "per_page": per_page,
        "sort": "relevance_score:desc",
        "filter": f"title_and_abstract.search:{query.replace(' ', '+')}"
    }
    time.sleep(6)
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()

    # Extract title and abstract
    records = []
    for result in data.get("results", []):
        title = result.get("title", "")
        abstract = result.get("abstract_inverted_index", "")
        # OpenAlex returns abstract as an inverted index; reconstruct if present
        if isinstance(abstract, dict):
            # Reconstruct abstract from inverted index
            abstract_words = sorted(abstract.items(), key=lambda x: min(x[1]))
            abstract_text = " ".join(word for word, _ in abstract_words)
        else:
            abstract_text = abstract or ""
        records.append({"title": title, "abstract": abstract_text})

    df = pd.DataFrame(records)
    return df


In [ ]:
# Example: Search for "agentic retrieval augmented generation"

#https://api.openalex.org/works?page=1&filter=title_and_abstract.search:agentic+retrieval+augmented+generation&sort=relevance_score:desc&per_page=10

df = search_openalex_all_pages(Search_qwuery)
print(len(df))  # Now this will be the total number of results
print(df.head())
#save df data frame
df.to_pickle("openalex_responses.pkl")


45
                                               title  \
0  Quantum Machine Learning in Feature Hilbert Sp...   
1  Adaptive machine learning framework to acceler...   
2             Machine Learning: Quantum vs Classical   
3  Enhancing Generative Models via Quantum Correl...   
4  Logical Approaches to Computational Barriers: ...   

                                            abstract  
0  A basic idea of quantum computing is surprisin...  
1  Quantum mechanics‐based ab initio molecular dy...  
2  Encouraged by growing computing power and algo...  
3  Generative modeling using samples drawn from t...  
4  The 12 papers in this special issue arose from...  


In [ ]:
CORE_API_KEY = "xxxxxxxxxxxxxxx"  # Replace with your CORE API key


def search_core(query, page=1, page_size=10):
    base_url = "https://api.core.ac.uk/v3/search/works"
    headers = {"Authorization": f"Bearer {CORE_API_KEY}"}
    params = {
        "q": query,
        "page": page,
        "pageSize": page_size,
        "fulltext": "true"  # Only return works with fulltext available
    }
    response = requests.get(base_url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def download_file(url, filename):
    time.sleep(6)
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

def core_results_to_df_and_download(results, output_dir):
    records = []
    for idx, work in enumerate(results.get("results", [])):
        title = work.get("title", "")
        abstract = work.get("abstract", "")
        download_url = work.get("downloadUrl", "")
        # Sanitize filename
        safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).rstrip()
        filename = f"{output_dir}/{idx}_{safe_title[:50]}.pdf"  # Adjust extension if needed
        records.append({
            "title": title,
            "abstract": abstract,
            "download_url": download_url,
            "filename": filename if download_url else ""
        })
        # Download the file if URL is present
        if download_url:
            try:
                download_file(download_url, filename)
                print(f"Downloaded: {filename}")
            except Exception as e:
                print(f"Failed to download {download_url}: {e}")
                records[-1]["filename"] = ""  # Mark as failed
    return pd.DataFrame(records)


In [ ]:
def search_core_page(query, page=1, page_size=10):
    base_url = "https://api.core.ac.uk/v3/search/works"
    headers = {"Authorization": f"Bearer {CORE_API_KEY}"}
    params = {
        "q": query,
        "page": page,
        "pageSize": page_size,
        "fulltext": "true"
    }
    time.sleep(6)
    response = requests.get(base_url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()
data = search_core_page(Search_qwuery)
print("Total hits:", data.get("totalHits", 0))
print("Results on this page:", len(data.get("results", [])))

Total hits: 23870
Results on this page: 4


In [ ]:
# Example usage
OUTPUT_DIR = "core_downloads"  # Folder to save downloaded files

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

results = search_core(Search_qwuery)
df = core_results_to_df_and_download(results, OUTPUT_DIR)
print(len(results))
print(df.head())

Downloaded: core_downloads/0_Quantum machine learning a classical perspective.pdf
Downloaded: core_downloads/1_A Quantum Computational Learning Algorithm.pdf
Downloaded: core_downloads/2_Natural evolution strategies and variational Monte.pdf
Downloaded: core_downloads/3_Micro-Data Learning The Other End of the Spectrum.pdf
5
                                               title  \
0  Quantum machine learning: a classical perspective   
1         A Quantum Computational Learning Algorithm   
2  Natural evolution strategies and variational M...   
3  Micro-Data Learning: The Other End of the Spec...   

                                            abstract  \
0  Recently, increased computational power and da...   
1  An interesting classical result due to Jackson...   
2  A notion of quantum natural evolution strategi...   
3  Many fields are now snowed under with an avala...   

                                download_url  \
0  https://core.ac.uk/download/154747677.pdf   
1      http://a

In [ ]:
def search_core_all_pages(query, per_page=10):
    base_url = "https://api.core.ac.uk/v3/search/works"
    headers = {"Authorization": f"Bearer {CORE_API_KEY}"}
    params = {
        "q": query,
        "page": 1,
        "pageSize": per_page,
        "fulltext": "true"
    }
    records = []
    while True:
        time.sleep(6)
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        if not data.get("results"):
            break
        records.extend(data["results"])
        params["page"] += 1
    return records

def download_file(url, filename):
    time.sleep(6)
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

def prepare_and_download(results, output_dir):
    all_records = []
    downloadable_records = []
    for idx, work in enumerate(results):
        title = work.get("title", "")
        abstract = work.get("abstract", "")
        download_url = work.get("downloadUrl", "")
        # Sanitize filename
        safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).rstrip()
        filename = f"{output_dir}/{idx}_{safe_title[:50]}.pdf" if download_url else ""
        record = {
            "title": title,
            "abstract": abstract,
            "download_url": download_url,
            "filename": filename
        }
        all_records.append(record)
        if download_url:
            downloadable_records.append(record)
            try:
                download_file(download_url, filename)
                print(f"Downloaded: {filename}")
            except Exception as e:
                print(f"Failed to download {download_url}: {e}")
                downloadable_records[-1]["filename"] = ""
    return pd.DataFrame(all_records), pd.DataFrame(downloadable_records)

results = search_core_all_pages(Search_qwuery)
df_all, df_downloadable = prepare_and_download(results, OUTPUT_DIR)

print("\033[1mAll results:\033[0m")
print(df_all)
print("\033[1mDownloadable results:\033[0m")
print(df_downloadable)
print(f"\033[1mTotal results:\033[0m {len(df_all)}")
print(f"\033[1mDownloadable results:\033[0m {len(df_downloadable)}")

HTTPError: 429 Client Error: Too Many Requests for url: https://api.core.ac.uk/v3/search/works/?q=Quantum+Algorithm+foundations+for+Machine+learning&page=3&pageSize=10&fulltext=true